In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
movies = pd.read_csv('./Datasets/peliculas.csv')

FileNotFoundError: ignored

In [0]:
movies.head()

In [0]:
movies.shape

In [0]:
movies.columns

In [0]:
movies.index

In [0]:
columna1 = movies['movie_title']
columna1.head()

In [0]:
linea = movies.loc[10, :]
linea

In [0]:
columna1 = movies['movie_title']
#Otra manera de obtener lo mismo podemos, pero de una manera al estilo "numpy"
columna1 = movies.loc[:,'movie_title']

In [0]:
columna1 = movies['movie_title']

In [0]:
movies.info()

### Separaremos los datos numéricos de los de texto

In [0]:
# columnas numéricas y columnas de texto
movies.dtypes

In [0]:
movies.dtypes == float

In [0]:
(movies.dtypes == "float64") | (movies.dtypes == "int64")

Ahora crear 2 listas distintas. Una  con los nombres de las columnas que son numéros y otra con los nombres de las columnas que son texto.


**Recuerda que todos estos objetos son iterables.**

In [0]:
num = (movies.dtypes == "float64") | (movies.dtypes == "int64")
num


In [0]:
num.index

In [0]:
for el in num:
    print(el)

In [0]:
for el in num.index:
    print(el)

In [0]:
num_cols = [c for c in num.index if num[c] ]
num_cols

In [0]:
num_col = [c for c in movies.columns if (movies[c].dtype == (float)) | (movies[c].dtype == ('int64'))]
num_col

In [0]:
obj_col = [c for c in movies.columns if (movies[c].dtype == object) ]
obj_col

### Inspección cuantitativa y de salud de los datos

In [0]:
movies_num = movies[num_col]

In [0]:
movies_num.describe()

In [0]:
#Estadísticas de las columnas

In [0]:
movies_num['duration'].hist()

In [0]:
movies_num['imdb_score'].hist()

In [0]:
movies_num['budget'].hist()

<div class="alert alert-block alert-info" style="margin-top: 20px">
Para poder debuggear esta situación utilizaremos una tecnica muy tipica de pandas que se llama <strong>boolean filtering</strong>. Basicamente construimos una serie de booleanos con el mismo indice que nuestro dataframe (la serie clasicamente la llamaremos mask).
</div>

In [0]:
mask = (movies_num['budget']> 1e9)

In [0]:
movies[mask]

* **"pd.Series.value_counts"** lo que hace es contar en una columna cuántos elementos hay del mismo valor. En este caso, tenemos True o False

In [0]:
financials =pd.read_csv('./Datasets/thenumbers.csv')
financials.head()

In [0]:

financials = financials[['movie_title', 'production_budget', 'worldwide_gross']]
financials.head()

**Dato:** Por defecto se concatena line por linea " axis = 0 ", pero si lo queremos es concatenar columna por columna escribimos " axis = 1 "  

In [0]:
movies_num =pd.concat([movies_num, movies['movie_title']], axis = 1) 
movies_num.head()

In [0]:
movies_v2 = pd.merge(financials, movies_num, on= 'movie_title', how ='left') 
movies_v2.head()


## Ahora solucionaremos el problema de los datos faltantes (nulos o NaN).

<div class="alert alert-success">
  Los datos faltantes generan problemas con muchos algoritmos de ML. Es por esto que existen distintas estrategias para lidiar con ellos.
</div>

* Hacemos la fusión de los dataframes "financials" y "movies_num" con **"pd.merge()"**
* Para ello le pasamos los 2 dataframes a juntar; luego, a  **"on"** le pasamos la  clave que nos va a relacionar ambos dataframes, porque aparece en ambas.
* A **"how"** le decimos que el largo debe de ser como el largo de las columnas de las claves de el dataset izquierdo **" *left* "**


In [0]:
movies_v2.notnull().head()
movies_v2

 * **"pd.Series.value_counts"** lo que hace es contar en una columna cuántos elementos hay del mismo valor. En este caso, tenemos True o False.

* **"apply"** es una función que nos evita hacer un for loop manualmente

* Ahora lo que va a hacer el "apply" es que columna por columna va a aplicar **"pd.Serie.value_counts"** y con ello obtendremos **cuántos elementos nulos tenemos en cada columna**


In [0]:
movies_v2.notnull().apply(pd.Series.value_counts) 

* Ahora haremos lo mismo que antes para poder ver las filas que al menos tengan un valor nulo o vacío
* Primero a available le pasamos todo el dataframe con indicaciones de true si es un valor no nulo o no vacío


In [0]:
available = ((movies_v2 !=0) & (movies_v2.notnull()))
available


* Ahora, mostraremos si en cada line tenemos al menos un valor nulo o vacío
  **"all"** es para que revise toda la linea y **"axis = 1"** es para que lo haga con cada "columna"

In [0]:
available.all(axis=1)

Con **".values_counts()"** veremos cuántas líneas tienen valores falsos

In [0]:
available.all(axis=1).value_counts()

* Vemos que hay 977 lineas con datos nulos o vacíos, por lo que debemos de limpiar nuestros datos para que no afecte a nuestro modelo. Sien embargo, perdeíamos muchos datos.
<div class="alert alert-success">  
Lo más importante es nuestra variable objetivo. De esta manera, si queremos eliminar algunas líneas que tengan un elemento vacío o nulo, esta debe de ser solo considerando a nuestra variable objetivo.Es decir, las columnas que no sean nuestra variable objetivo y que en alguna línea tenga un elemento vacío deberemos de llenarla con el promedio u otro dato representativo.  
    
</div>

* No podemos entrenar nuestro algoritmo con datos cuya variable objetivo no esta definida o sea nula (valor falso). Eliminemos esas líneas.



In [0]:
mask = available['worldwide_gross']
print(mask.shape)
mask


In [0]:
((mask != 0) & (mask.notnull())).value_counts()

Lo que haremos en la siguiente celda es que vamos a reducir el número de líneas del dataframe a solo el número de líneas que no sean vacías o nulas de nuestra variable objetivo **worldwide_gross**

In [0]:
movies_v2 = movies_v2[mask]
print(movies_v2.shape)
movies_v2


Ahora corroboraremos que justamente nos queda solo los valores que no son vacíos de nuestra variable objetivo.

In [0]:
((movies_v2 != 0) & (movies_v2.notnull())).worldwide_gross.value_counts()

<div class="alert alert-success">
  En el caso de las features que no son la variable objetivo una mejor solución para lidiar con los datos faltantes es <strong>remplazar estos datos por otros que sean manejables</strong> y no afecten la calidad de las predicciones. La estrategia más comun es utilizar la media de todos los ejemplos para la feature dada.
</div>

* Reconoceremos a los datos faltantes como "missing_values =** np.nan **"
* Ahora reemplazaremos esos datos faltantes conel promedio. Para ello, hacemos " strategy =**'mean'** "

* Con **" axis=1 "** hacemos que tome la media de cada columna, *pero en este caso lo estamos omitiendo  porque no estamos utilizando el mismo método del profesor*

In [0]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')

* Debido a que con el método "fit_transform" solo vamos a trabajar con datos numéricos, vamos a dejar la columna "movie_title".
* Además, podemos observar que tenemos 2 columnas con datos iguales : " duration " y " duration.1 "
* Utilizaremos el método **.drop()**

In [0]:
movies_v2 = movies_v2.drop('movie_title', axis=1)

In [0]:
#Ahora dejaremos la columna duplicada
movies_v2 =movies_v2.drop('duration', axis=1)
movies_v2.head()

In [0]:
values = imputer.fit_transform(movies_v2)
values

* Acabamos de obtener un arreglo.
* Ahora procederemos a volver a convertirlo a un DataFrame

In [0]:
X = pd.DataFrame(values)
X

In [0]:
X.columns = movies_v2.columns
X

#### Muy importante:
Los índices también necesitan ser modificados, ya que el index de movies_v2 indicaba la posición en la lista de cada película.
Con ello obteníamos índices desde el 1 hasta el 4384;sin embargo, en el nuevo DataFrame tenemos índices desde el 1 hasta el 4103, lo cual nos indica que hemos alterado la posición de cada línea o película.

In [0]:
print(movies_v2.index)

In [0]:
print(X.index)

In [0]:
print(movies_v2.index == X.index)

In [0]:
X.index = movies_v2.index
X

Ahora lo guardaremos como archivo "csv"

In [0]:
x.to_csv('./Datasets/X.csv', index='False')